In [1]:
using StaticArrays
using BenchmarkTools
include("../src/Granulars.jl")

In [ ]:
p1 = Particle(r=[0.0,0.0,0.0])
p2 = Particle(r=[0.9,0,0])

In [ ]:
c = Config(0.001, 10.0)

In [ ]:
beam = Beam(p1,p2,c) # falta la orientación de la otra esfera

In [ ]:
Beam_Orientation(unitary(p2.r - p1.r ))

In [ ]:
quat_to_angle(p1.q)

In [ ]:
quat_to_angle(p1.q)-Beam_Orientation(unitary(p2.r - p1.r ))

In [ ]:
θ_beam = quat_to_angle(p1.q)-beam.Δθ_0

In [ ]:
Δθ = quat_to_angle(p2.q) * quat_to_angle(p1.q) * beam.Δθ_0ij

In [ ]:
@btime Beam($p1,$p2,$c)

In [ ]:
Beam_Orientation(normalize(SVector(1.0,1.0,1.0)), :ZYX)

In [ ]:
@btime Beam_Orientation(SVector(1.0,1.0,1.0), :ZYX)

In [ ]:
Lab_to_Beam(normalize(SVector(1.0,1.0,1.0)), SVector(0.785398, -0.61548, -8.261799))

In [ ]:
Beam_to_Lab(SVector(1.0,0.0,0.0), SVector(0.785398, -0.61548, 4.2661799))

In [ ]:
v = SVector(1.0,1e-14,0.0)

In [ ]:
v = [1.0,1e-14,0.0]
v[abs(v) .< 1e-12] = 0.0

In [ ]:
sparsify(v, eps) = abs(v) < eps ? 0.0 : v

function test(v)
    v = sparsify.(v,1e-12)
    v
end

In [ ]:
@btime test($v)

In [2]:
using NLsolve

In [ ]:
function f!(F, x)
    F[1] = (x[1]+3)*(x[2]^3-7)+18
    F[2] = sin(x[2]*exp(x[1])-1)
end

In [ ]:
nlsolve(f!, [ 0.1; 1.2], autodiff = :forward)

In [ ]:
@btime nlsolve($f!, $[0.1; 1.2], autodiff = :forward)

In [ ]:
function f!(F, x)
    F[1] = (x[1]+3)*(x[2]^3-7)+18
    F[2] = sin(x[2]*exp(x[1])-1)
end

function j!(J, x)
    J[1, 1] = x[2]^3-7
    J[1, 2] = 3*x[2]^2*(x[1]+3)
    u = exp(x[1])*cos(x[2]*exp(x[1])-1)
    J[2, 1] = x[2]*u
    J[2, 2] = u
end

@btime  nlsolve($f!, $j!, $[ 0.1; 1.2])

In [ ]:
function func(F, w)
    τ = [1.0,1.0,1.0]
    II = [1.0,1.0,1.0]
    dt = 0.01
    F[1] = w[1] + (τ[1] + w[2]*w[3]*(II[2]-II[3]))*dt/II[1]
    F[2] = w[2] + (τ[2] + w[3]*w[1]*(II[3]-II[1]))*dt/II[2]
    F[3] = w[3] + (τ[3] + w[1]*w[2]*(II[1]-II[2]))*dt/II[3]
end

In [ ]:
sol = nlsolve(func, [ 0.1; 1.2; 1.3], autodiff = :forward)

In [ ]:
SVector{3}(sol.zero)

In [39]:
function Move_w(w::SVector{3, Float64}, τ::SVector{3, Float64}, II::SVector{3, Float64}, dt::Float64, cte=1.0::Float64)
    
    function func(x, F)        
        F[1] = -x[1] + w[1] + (τ[1] + 0.5*(w[2]*w[3] + x[2]*x[3])*(II[2]-II[3]))*dt*cte/II[1]
        F[2] = -x[2] + w[2] + (τ[2] + 0.5*(w[3]*w[1] + x[3]*x[1])*(II[3]-II[1]))*dt*cte/II[2]
        F[3] = -x[3] + w[3] + (τ[3] + 0.5*(w[1]*w[2] + x[1]*x[2])*(II[1]-II[2]))*dt*cte/II[3]
    end
    
    df = OnceDifferentiable(func, [w[1]; w[2]; w[3]], similar([w[1]; w[2]; w[3]]))
    SVector{3}(nlsolve(df, [w[1]; w[2]; w[3]]).zero)
end

Move_w (generic function with 2 methods)

In [65]:
Move_w(SVector(1.0,1.0,1.0), SVector(1.0,0.0,0.0), SVector(1.0,1.1,1.0), 0.001)

3-element SVector{3, Float64} with indices SOneTo(3):
 1.00105
 1.0
 0.99995

In [92]:
function Move_w(w::SVector{3, Float64}, τ::SVector{3, Float64}, II::SVector{3, Float64}, dt::Float64, cte=1.0::Float64)::SVector{3, Float64}
    ww::SVector{3, Float64} = w
    
    for i in 1:30
        ww = w + SVector(
            τ[1] + 0.5*(w[2]*w[3]+ww[2]*ww[3])*(II[2]-II[3]),
            τ[2] + 0.5*(w[3]*w[1]+ww[3]*ww[1])*(II[3]-II[1]),
            τ[3] + 0.5*(w[1]*w[2]+ww[1]*ww[2])*(II[1]-II[2]))*dt*cte./II
        
        println(norm(ww-w))
    end
    ww
end

Move_w (generic function with 2 methods)

In [103]:
Move_w(SVector(1.0,1.0,1.0), SVector(1.0,10.0,1.0), SVector(2.3,1.2,1.3), 0.0001)

0.000768196501868215
0.0007681952591023242
0.0007681952577299909
0.0007681952577299909
0.0007681952577299909
0.0007681952577299909
0.0007681952577299909
0.0007681952577299909
0.0007681952577299909
0.0007681952577299909
0.0007681952577299909
0.0007681952577299909
0.0007681952577299909
0.0007681952577299909
0.0007681952577299909
0.0007681952577299909
0.0007681952577299909
0.0007681952577299909
0.0007681952577299909
0.0007681952577299909
0.0007681952577299909
0.0007681952577299909
0.0007681952577299909
0.0007681952577299909
0.0007681952577299909
0.0007681952577299909
0.0007681952577299909
0.0007681952577299909
0.0007681952577299909
0.0007681952577299909


3-element SVector{3, Float64} with indices SOneTo(3):
 1.0000391284528594
 1.000749991637224
 1.00016157184863

In [78]:
@btime Move_w(SVector(1.0,1.0,1.0), SVector(1.0,0.0,0.0), SVector(1.0,1.1,1.0), 0.001)

  165.565 ns (0 allocations: 0 bytes)


3-element SVector{3, Float64} with indices SOneTo(3):
 1.00109999499725
 1.0
 0.9998999450002501